<a href="https://colab.research.google.com/github/Zinkonaing-John/SLM/blob/master/SLM_fine-turned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
!pip install transformers accelerate

In [88]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [89]:
training_data = [
  {
    "input": "Iris Dataset",
    "output": "A classic, small, multivariate dataset for classification, containing 150 instances of iris flowers with four features (sepal length, sepal width, petal length, and petal width) across three species."
  },
  {
    "input": "MNIST Dataset",
    "output": "A large database of 70,000 images of handwritten digits (0 through 9), primarily used for image classification and deep learning practice."
  },
  {
    "input": "Titanic Dataset",
    "output": "Contains data about the passengers on the Titanic, including age, gender, ticket class, and whether they survived. It is a fundamental dataset for binary classification."
  },
  {
    "input": "Boston Housing Dataset",
    "output": "Includes information about houses in the Boston area (e.g., crime rate, number of rooms) with the goal of predicting the median house price. It is a common dataset for regression analysis."
  },
  {
    "input": "Credit Card Fraud Detection Dataset",
    "output": "Transaction data made by European cardholders, highly unbalanced with a small number of fraudulent transactions. Used for anomaly detection and fraud prediction."
  },
  {
    "input": "ImageNet",
    "output": "An extremely large visual database with millions of annotated images across thousands of categories. Used for training large-scale deep learning models for image recognition and object detection."
  },
  {
    "input": "Red Wine Quality Dataset",
    "output": "Contains physicochemical data for Portuguese 'Vinho Verde' wines, with a quality rating. Used for regression or classification to predict wine quality."
  },
  {
    "input": "U.S. Chronic Disease Indicators (CDI)",
    "output": "A comprehensive set of indicators from the CDC covering health status, risk factors, and health care access related to chronic diseases. Used for public health analysis."
  },
  {
    "input": "MovieLens Datasets",
    "output": "Collections of movie ratings from users, which are essential for building and testing recommender systems."
  },
  {
    "input": "Common Crawl Corpus",
    "output": "An open repository of petabytes of raw web page data and text extractions from web crawls. It is widely used for large-scale text mining and natural language processing (NLP) research."
  }
]

print("data is ready!")

data is ready!


In [90]:
input_text = "Iris Dataset"

input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids)

decode_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Before fine-tuning: {decode_output}")

Before fine-tuning: Iris Dataset


In [91]:
!pip install datasets
from datasets import Dataset


In [92]:
training_data_list =[
  {
    "input": "Iris Dataset",
    "output": "A classic, small, multivariate dataset for classification, containing 150 instances of iris flowers with four features (sepal length, sepal width, petal length, and petal width) across three species."
  },
  {
    "input": "MNIST Dataset",
    "output": "A large database of 70,000 images of handwritten digits (0 through 9), primarily used for image classification and deep learning practice."
  },
  {
    "input": "Titanic Dataset",
    "output": "Contains data about the passengers on the Titanic, including age, gender, ticket class, and whether they survived. It is a fundamental dataset for binary classification."
  },
  {
    "input": "Boston Housing Dataset",
    "output": "Includes information about houses in the Boston area (e.g., crime rate, number of rooms) with the goal of predicting the median house price. It is a common dataset for regression analysis."
  },
  {
    "input": "Credit Card Fraud Detection Dataset",
    "output": "Transaction data made by European cardholders, highly unbalanced with a small number of fraudulent transactions. Used for anomaly detection and fraud prediction."
  },
  {
    "input": "ImageNet",
    "output": "An extremely large visual database with millions of annotated images across thousands of categories. Used for training large-scale deep learning models for image recognition and object detection."
  },
  {
    "input": "Red Wine Quality Dataset",
    "output": "Contains physicochemical data for Portuguese 'Vinho Verde' wines, with a quality rating. Used for regression or classification to predict wine quality."
  },
  {
    "input": "U.S. Chronic Disease Indicators (CDI)",
    "output": "A comprehensive set of indicators from the CDC covering health status, risk factors, and health care access related to chronic diseases. Used for public health analysis."
  },
  {
    "input": "MovieLens Datasets",
    "output": "Collections of movie ratings from users, which are essential for building and testing recommender systems."
  },
  {
    "input": "Common Crawl Corpus",
    "output": "An open repository of petabytes of raw web page data and text extractions from web crawls. It is widely used for large-scale text mining and natural language processing (NLP) research."
  }
]

In [93]:
training_dataset = Dataset.from_list(training_data_list)


def tokenize_function(examples):
   inputs = ["Answer the questions: " + doc for doc in examples ["input"]]

   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
   labels = tokenizer(text_target=examples["output"], max_length=128, truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs


tokenized_dataset = training_dataset.map(tokenize_function, batched=True)

print("Dataset has been tokenized!")
print(tokenized_dataset)



Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset has been tokenized!
Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})


In [94]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq


trainging_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,  # Increased epochs
    per_device_train_batch_size=2,
    learning_rate=0.001, # Decreased learning rate
    logging_steps=10,
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [95]:
trainer = Trainer(
    model=model,
    args=trainging_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Fine-turining has started...")
trainer.train()
print("Fine-turining has finished!")

Fine-turining has started...


Step,Training Loss
10,3.492700
20,1.503100
30,0.559200
40,0.272100
50,0.197400
60,0.148500
70,0.068800
80,0.055300
90,0.053200
100,0.049200


Fine-turining has finished!


In [96]:
import torch
torch.manual_seed(42)

In [98]:
input_text = "ImageNet"

prompt = "Give related data : " + input_text
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Move the input tensor to the same device as the model
device = model.device
input_ids = input_ids.to(device)


outputs = model.generate(input_ids, max_new_tokens=50)

decode_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("---" * 10)
print(f"input: {input_text}" )
print(f"After fine-tuning: {decode_output}")

------------------------------
input: ImageNet
After fine-tuning: An extremely large visual database with millions of annotated images across thousands of categories. Used for training large-scale deep learning models for image recognition and object detection.
